<a href="https://colab.research.google.com/github/Dorin-Irimia/LLM_Google_Colab/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Antrenarea unui LLM pentru a putea redacta teste pe baza unor cerinte.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Instalarea Pachetelor Necesare

In [2]:
# Instalează dependințele necesare
!pip install torch transformers peft bitsandbytes accelerate datasets pandas openpyxl


  Using cached bitsandbytes-0.45.3-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached datasets-3.3.2-py3-none-any.whl.metadata (19 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x8

## Îmbinarea Dataset-urilor (Requirements + Tests)

In [17]:
import pandas as pd
import json

# ✅ Încarcă fișierele Excel
req_file = "Requirements.xlsx"
test_file = "Test_cases.xlsx"

df_req = pd.read_excel(req_file)
df_tests = pd.read_excel(test_file)

# ✅ Convertim numele coloanelor pentru consistență (eliminăm spații suplimentare)
df_req.columns = df_req.columns.str.strip()
df_tests.columns = df_tests.columns.str.strip()

# ✅ Unim requirement-urile cu testele pe "Requirement ID"
df_merged = df_tests.merge(df_req, on="ID_REQ", how="left")

# ✅ Conversie la JSONL pentru fine-tuning
data = []
for _, row in df_merged.iterrows():
    prompt = f"ID_REQ: {row['ID_REQ']}\nGenerate a system test for this requirement:"
    response = f"System Test: {row['Test Case']} (covers requirement ID: {row['ID_REQ']})"
    data.append({"prompt": prompt, "response": response})

# ✅ Salvăm dataset-ul final
jsonl_path = "dataset.jsonl"
with open(jsonl_path, "w", encoding="utf-8") as f:
    for entry in data:
        f.write(json.dumps(entry) + "\n")

print(f"✅ Dataset-ul pentru AI a fost creat: {jsonl_path}")


KeyError: 'ID_TEST'

## Încărcarea Dataset-ului (Excel → JSONL)


In [3]:
import pandas as pd
import json

# Încarcă fișierul Excel
df = pd.read_excel("dataset.xlsx")

# Conversie la JSONL
data = []
for _, row in df.iterrows():
    prompt = f"Requirement: {row['Requirement']}\nGenerate a system test for this requirement:"
    response = f"System Test: {row['Test Case']} (covers requirement ID: {row['Requirement ID']})"
    data.append({"prompt": prompt, "response": response})

# Salvare dataset
with open("dataset.jsonl", "w", encoding="utf-8") as f:
    for entry in data:
        f.write(json.dumps(entry) + "\n")

print("✅ Dataset salvat ca dataset.jsonl!")


✅ Dataset salvat ca dataset.jsonl!
